# Table of Contents
 1. Installation & Setup
 2. Core Agent Components
 3. Agent Types
 4. Tools & Toolkits
 5. Memory Systems
 6. Prompt Engineering  for Agents
 7. Agent Execution Patterns
 8. Multi-Agent Systems
 9. Advance Patterns
 10. Debugging & Monitoring
 11. Production Considerations



# Installation & Setup

In [ ]:
# Core installation
!pip install langchain langchain-openai langchain-community

# Additional packages for specific tools
pip install langchain-experimental
pip install duckduckgo-search
pip install wikipedia
pip install requests beautifulsoup4
pip install pandas numpy
pip install python-dotenv

#Basic Setup
import os
from dotenv import load_dotenv
from langchain_openai imoprt ChatOpenAI
from langchain.agents import initialize_agent, AgentType

load_dotenv()
llm = ChatOpenAI(temperature=0, model="gpt-5")

# Core Agent Components

## 1. Language Model (LLM)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

# OpenAI
llm = ChatOpenAI(
    model="gpt-5",
    temperature=0,
    max_tokens=1000,
)

# Anthropic Claude
llm = ChatAnthropic(
    model = "claude-3-sonnet-20240229"
    temperature=0
)

# Google Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0
)

## 2. Tools

In [ ]:
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Simple function-based tool
def calculator(expression: str) -> str:
  """ Calculate mathematical expressions"""
  try:
    return str(evel(expression))
  except:
    return "Invalid Expression"

calculator_tool = Tool(
    name = "Calculator",
    func="calculator",
    description = "Useful for mathematical calculations. Input should be valid mathematical expression."
)

# Pre-built tools
search = DuckDuckGoSearchRun()
wikipedia=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [calculator_tool, searcch, wikipedia]

## Agent Initialziation

In [ ]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    max_iterations=3,
    early_stopping_method = "generate",
)

# Agent Types

## 1. Zero-Shot ReAct Agent


In [ ]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    max_iterations=3,
    early_stopping_method = "generate",
)

#Usage
result = agent.run("What's the weather in Bahawalpur.")

## 2. Conversational ReAct Agent

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.memory improt ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.CONVERSATIONS_REACT_DESCRIPTION,
    memory = memory,
    verbose = True,
)

## 3. Structured Tool Chat Agent


In [ ]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.STRUCTURE_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
)

## 4. OpenAI Functions Agent

In [ ]:
from langchain.agents import initialze_agent, AgentType

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent= AgentType.OPENAI_FUNCTIONS,
    verbose = True,
)

## 5. Custom Agnet with LangGraph

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

class AgentState(TypedDict):
  messages: List[str]
  next_action: str

def should_continue(state):
  return "continue" if len(state(["messages"])) < 5 else "end"

def call_model(state):
  # Your model calling logic
  response = llm.invoke(state["messages"][-1])
  return {"messages":state["messages"] + [response.content]}

workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_conditional_edges("agent", should_continue, {
    "continue": "agent",
    "end": END
})
workflow.set_entry_point("Agent")

app = workflow.compile()


# Tools and Toolkits

## 1. Custom Tools

In [ ]:
from langchain.tools import BaseTool
from typing import Optional
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
  expression: str = Field(description="Mathematical expression to evaluate.")

class CalculatorTool(BaseTool):
  name = "calculator"
  description="useful for mathematical calculations"
  args_schema = CalculatorInput

  def _run(self, expression: str) -> str:
    try:
      return str(eval(expression))
    except Exception as e:
      return f"Error: {str(e)}"
  async def _arun(self, expression: str) -> str:
    return self._run(expression)

calculator = CalculatorTool()

## 2. File System Tools


In [ ]:
from langchain_community.tools import ReadFileTool, WriteFileTool
read_tool = ReadFileTool()
write_tool = WriteFileTool()

tools = [read_tool, write_tool]

## 3. Web Scraping Tools

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.tools import Tool

def scrape_website(url: str) -> str:
  """Scrape content from a webiste"""
  try:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()[:5000]
  except Exception as e:
    return f"Error scraping {url}: {str{e}}"

scraper_tool = Tool(
    name="WebScraper",
    func=scrape_website,
    description="Scrape content from websites. Input should be valid URL."
)

## 4. Database Tools

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.utilities import SQLDatabase

# SQLite
db = SQLDatabase.from_url("sqlite://example.db")
db_tool = QuerySQLDataBaseTool(db=db)

tools = [db_tool]

## 5. API Tools

In [ ]:
import requests
from langchain.tools import Tool
def call_api(endpoint: str) -> str:
  """Make API calls to external services."""
  try:
    response = requests.get(endpoint)
    return response.json()
  except Exception as e:
    return f"API Error: {str(e)}"

  api_tool = Tool(
      name="APITool",
      func = call_api,
      description = "Make API calls. Input should be a valid API endpoint."
  )

# Memory Systems

## 1. Conversation Buffer Memory


In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True
)

## 2. Conversation Buffer Window Memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    k = 5,
    memory_key = "chat_history",
    return_messages = True,
)

## 3. Conversation Summary Memory

In [ ]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm = llm,
    memory_key = "chat_history",
    return_messages = True,
)

## 4. Vector Store Memory

In [ ]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts([""], embeddings)
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))

memory = VectorStoreRetrieverMemory(
    retriever = retriever,
    memory_key = "chat_history",
)

## 5. Custom Memory



In [ ]:
from langchain.memory.chat_memory import BaseChatMemory
from langchain.schema import BaseMessage

class CustomMemory(BaseChatMemory):
  def __init__(self):
    super().__init__(return_messages=True)
    self.storage = []

  def save_context(self, inputs, outputs):
    # Custom save logic
    self.storage.append({"input":inputs, "output":outputs})

  def clear(self):
    self.storage = []

# Prompt Engineerig for Agents

## 1. Custom Agent Prompts


In [ ]:
from langchain.agents import ZeroShotAgent
from langchain.prompts import PromptTemplate

template = """

You are a helpful AI assistant with access to the following tool:
{tools}

Use the following format:

Question: the input question you must answer.
Thought: You should always think about what to do.
Action: The action to take, should be none of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
...(this Thought/Action/Action Input/Observation can repeat  N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpage}
"""

prompt = ZeroShotAgent.create_prompt(
    tools = tools,
    prefix = "You are a helpful AI Assistant.",
    suffix="Begin!\n\nQuestion: {input}\n{agent_scratchpage}",
    input_variables = ["input","agent_scratchpage"],
)

agent = ZeroShotAgent(
    llm_chain = llm,
    tools = tools,
    prompt = prompt
)


## 2. System Message Customization


In [ ]:
from langchain.agents import AgentType, initialze_agent
from langchian.memory import ConversationBufferMemory

system_message = """
You are an expert data analyst AI assistant. You have access to the various tools to help analyze data, search for information, and perform calculations.

Always:
1. Think step by step
2. Verify your calculations
3. Provide clear explanations
4. Ask for clarification if the request is ambiguous

When working with data:
- Always examine data structure first
- Handle missing values appropriately
- Provide visualizations when helpful
"""

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgnetType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose = True,
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    agent_kwargs= {
        "system_message": system_message
    }

)

# Agent Execution Patterns

# 1. Basic Agent Execution

In [ ]:
# Simple Run
result = agent.run("What's 25*4 +10?")

# Run with Input
result = agent({"input": "Search for recent news about AI."})

# Async execution
import asyncio

async def run_agent():
  result = await agent.run("What's weather in Bahwalpur?")
  return result
result = asyncio.run(run_agent())

## 2. Streaming Agent Responses

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

## 3. Agent with Error Handling

In [ ]:
def safe_agent_run(agent, query, max_retries=3):
  for attempt in range(max_retries):
    try:
      result agent.run(query)
      return result
    exception Exception as e:
      print(f"Attempt {attempt + 1} failed: {ste(e)}")
      if attempt == max_retries-1:
        return f"Agent failed after{max_retries} attempts: {str(e)}"

result = sage_agent_run(agent, "Complex query here")

## 4. Agent with Timeout

In [ ]:
import signal
from contextlib import contextmanager

@contextmanager
def timeout(duration):
  def timeout_handler(signum, frame):
    raise TimeoutError(f"Timed out after {duration} seconds")

  signal.signal(signal.SIGALRM, timeout_handler)
  signal.alarm(duration)
  try:
    yield
  finally:
    signal.alarm(0)

try:
  with timeout(30):
    result = agent.run("Long running query")
  except TimeoutError:
    result "Agent Execution timedout."

# Multi-Agent Systems

## Sequential Agent Chain

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.schema import HumanMessage

# Create specialized agents
researcher = initialize_agent(
    tools = [search, wikipedia],
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

analyst = initialize_agent(
    tools = [calculator_tool],
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Sequential Execution
def multi_agent_workflow(query):
  # Step 1 : Research
  research_result = researcher.run(f"Research information about: {query}")

  # Step 2 : Analysis
  anaylysis_result = analyst.run(f"Analyze this information: {research result}")

  return {"research":research_result, "analysis":analysis_result}

result = multi_agent_workflow("Tesla's quarterly earnings.")

## 2. Parallel Agent Execution

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

async def parallel_agents(quries):
  with ThreadPoolExecutor() as executor:
    loop = asyncio.get_event_loop()
    tasks = [
        loop.run_in_executor(executor, agent.run, query)
    ]
    results = await asyncio.gather(*tasks)
  return results

queries = [
    "What's weather in Bahawalpur?",
    "Calculate 15 * 23 + 45",
    "Search for recent AI news"
]

results = asyncio.run(parallel_agents(queries))

## 3. Agent Conversation

In [ ]:
class AgentConversation:
  def __init__(self, agents):
    self.agents = agents
    self.conversation_history = []

  def run_conversation(self, initial_message, max_turns=5):
    current_message = initial_message
    current_agent_idx = 0

    for turn in range(max_turns):
      agent = self.agents[current_agent_idx]
      response = agent.run(current_message)

      self.conversation_history.append({
          "agent": current_agent_idx,
          "input": current_message,
          "output": response
      })

      # Switch to next agent
      current_agent_idx = (current_agent_idx + 1) % len(self.agents)
      current_message = response
    return self.conversation_history

# Usage
conversation = AgentConversation([researcher, analyst])
history = conversation.run_conversation("Analyze the impact of AI boom on job markets.")

# Advanced Patterns


## 1. Self-Correcting Agent

In [ ]:
class SelfCorrectingAgent:
  def __init__(self, agent, validator_llm):
    self.agent = agent
    self.validator = validator_llm

  def run_with_validation(self, query, max_connections=3):
    for attemp in range(max_connections):
      result = self.agent.run(query)

      # Validate result
      validation_prompt = f"""
      Query: {query}
      Result: {result}

      Is this result accurate and complete? If not, what corrections are needed?
      Respond with 'Valid' if correct, or 'Invalid: [explanation]' if incorrect.
      """

      validation = self.validator.invoke(validation_prompt).content

      if validation.startswith('Valid'):
        return result
      else:
        # Modify query for correction
        query = f"{query}\n\nPrevious attempt: {result}\nCorrection needed: {validation}"

     return result

correcting_agent = SelfCorrectingAgent(agent, llm)
result = correcting_agent.run_with_validation("Calculate.")

## 2. Agent with Planning

In [ ]:
from langchain.agents import PlanAndExecute
from langchain_experimental.plan_and_execute import (PlanAndExecuteAgentExecutor, load_agent_executor, load_chat_planner)

planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)

plan_ang_execute_agent = PlanAndExecuteAgentExecutor(
    planner = planner,
    executor = executor,
    verbose = true,
)

result = plan_and_execute_agent.run("Plan a 3-day trip to Paris including budget analysis.")

## 3. Hierarchical Agent System

In [ ]:
class HierarchicalAgent:
  def __init__(self, coordinator_agent, specialist_agents):
    self.coordinator = coordinator_agent
    self.specialists = specialist_agents

  def run(self, query):
    decision_prompt = f"""
    Query : {query}
    Available specialists: {list(self.specialists.keys())}
    Which specialist should handle this query? Respond with just the specialist name.
    """

    specialist_choice = self.coordinator.run(decision_prompt)

    # Route to appropriate specialist
    if specialist_choice in self.specialists:
      return self.specialists[specialist-choice].run(query)
    else:
      return self.coordinator.run(query)

# Create hierarchical system
specialists = {
    "math": initialize_agent([calculator_tool], AgentType.ZERO_SHOT_REACT_DESCRIPTION),
    "research": initialize_agent([search, wikipedia])m llm, AgentType.ZERO_SHOT_REACT_DESCRIPTION),
    "data": initialze_agent([db_tool], llm, AgentType.ZERO_SHOT_REACT_DESCRIPTION)
}

hierarchical_system = HierarchicalAgent(agent, specialsists)

# Debugging & Monitoring

## 1. Verbose Logging

In [ ]:
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Enable Verbose mode
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
)

# Run with detailed output
result = agent({"input": "What is 2+2"})
print("final result", result["output"])
print("intermediate steps", result["intermediate_steps"])

## 2. Custom Callbacks

In [ ]:
from langchain.callbacks.base import BaseCallBackHandler
from typing import Any, Dict, List

class CustomCallbackhandler(BaseCallbackHandler):
  def on_agent_action(self, action, **kwargs):
    print(f"Agent is taking action: {action.tool}")
    print(f"Agent input: {action.tool_input}")

  def on_agent_finish(self, finish, **kwargs):
    print(f"Agent finised with output: {finish.return_values}")

  def on_tool_start(self, serialized: Dict[str, Any], input_str: str, **kwargs):
    print(f"Tool {serialized["name"]} started with input {input_str}")

  def on_tool_end(self, output:str, **kwargs):
    print(f"Tool finished with output: {output}")

# Use custom callback
callback_handler = CustomCallbackHandler()
agent = initialzie_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callbacks=[callback_handler]
)

## 3. Performance Monitoring

In [ ]:
import time
from functools import wraps

def monitor_performance(func):
  @wraps(func)
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()

    print(f"Function {func.__name__} took {end_time-start_time:.2f} seconds")
    return result
  return wrapper

@monitor_performance
def run_agent_with_monitoring(query):
  return agent.run(query)

result = run_agent_with_monitoring("Complex query here.")

## 4. Error Tracking

In [ ]:
class ErrorTrackingAgent:
  def __init__(self, agent):
    self.agent = agent
    self.errors = []

  def run(self, query):
    try:
      result = self.agent.run(query)
      return result
    except Exception as e:
      error_info = {
          "query":query,
          "error":str(e),
          "timestamp":time.time()
      }
      self.errors.append(error_info)
      raise e

  def get_error_summary(self):
    return {
        "total_errors": len(self.errors),
        "recent_errors": self.errors[-5:] if self.errors else []
    }

tracked_agent = ErrorTrackingAgent(agent)

# Production Consideration

## 1. Rate Limiting

In [ ]:
import time
from threading import Lock

class RateLimitedAgent:
  def __init__(self, agent, calls_per_minute=60):
    self.agent = agent
    self.calls_per_minute = calls_per_minute
    self.calls = []
    self.lock = Lock()

  def run(self, query):
    with self.lock:
      now = time.time()
      # Remove calls older than 1 minute
      self.calls = [call_time for call_time in self.calls if now-call_time<60]

      if len(self.calls) >= self.calls_per_minute:
        sleep_time = 60 - (now - self.calls(0))
        time.sleep(sleep_time)

      self.calls.append(now)

    return self.agent.run(query)

rate_limited_agent = RateLimitedAgent(agent, calls_per_minute)

## 2. Caching Results

In [ ]:
from functools import lru_cache
import hashlib
import json

class CachedAgent:
  def __init__(self, agent, cache_size=128):
    self.agent = agent
    self.cache = {}
    self.cache_size = cache-size

  def _get_chache_key(self, query):
    return hashlib.md5(query.encode()).hexdigest()

  def run(self, query):
    cache_key = self._get_cache_key(query)

    if cache_key in self.cache:
      print("cache hit!")
      return self.cache[cache_key]
    result = self.agent.run(query)

    # Simple LRU cache Implementation
    if len(self.cache) >= self.cache-size:
      # Remove oldest entry
      oldest_key = next(iter(self.cache))
      del.self.cache[oldest_key]

    self.cache[cache_key] = result
    return result

cached_agent = CachedAgent(agent)

## 3. Configuration Management

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class AgentConfig:
  model_name: str "gpt-5"
  temperature: float = 0
  max_tokens: int = 1000
  max_iterations: int = 3
  verbose: bool = True
  timeout; int = 30
  tools: List[str] = "buffer"
  memory_size: int = 10

def create_agent_from_config(config: AgentConfig):
  llm = ChatOpenAI(
      model = config.model_name,
      temperature= config.temperature,
      max_tokens = config.max_tokens,
  )

  # configure memory based on type
  if config.memory_type == "buffer":
    memory = ConversationBufferMemory(memory_key="chat_history")
  elif config.memory_type = "window":
    memory = ConversationBufferWindowMemory(k=config.memory_size,memory_key="chat_history")
  else:
    memroy = None

  return initialze_agent(
      tools = tools,
      llm = llm,
      agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
      memory= memory,
      verbose=config.verbose,
      max_iterations = config.max_iterations
  )

# Usage
config = AgentConfig(model_name="gpt-5", temperature=0.2)
production_agent = create_agent_from_config(config)

## 4. Health Checks

In [ ]:
class HealthCheckedAgent:
  def __init__(self, agent):
    self.agent = agent
    self.is_healthy = True
    self.last_health_check = time.time()

  def health_check(self):
    try:
      # Simple health check with basic query
      result = self.agent.run("What is 1+1")
      self.is_healthy = "2" in str(result)
      self.last_health_check = time.time()
      return self.is_healthy
    except Exception:
      self.is_healthy = False
      return False

  def run(self, query):
    # Check health every 5 minutes
    if time.time() - self.last_health_check > 300:
      if not self.health_check():
        raise Exception("Agent Health Check failed.")
    if not self.is_healthy:
      raise Exception("Agent is not healthy")
    return self.agent.run(query)

healthy_agent = HealthCheckedAgent(agent)

# Quick Reference

In [ ]:
# Basic agent setuo
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun

llm = ChatGoogleGenerativeAI(model="gemini-flash", temperature="0")
tools = [DuckDuckGoSearchRun()]
agent = initialize_agent(tools, llm, AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Run Agent
result = agent.run("Your query here.")

# With memory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(tools, llm, AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory)

# Custom Tool
from langchain.tools import Tool
def my_function(input_str): return f"Result: {input_str}"
my_tool = Tool(name="MyTool", func=my_function, description="description here")

# Async execution
result = await agent.arun("Your query here.")

# What can I build with this

## >> Research Assistants
- Agents that search , analyse, and synthesize the information

## >> Data Analysis Bots
- Agents that query databases and perform calculations

## >> Task Automation
- Multi-step workflow agents with planning capabilities

## >> Customer Service Bots
- Conversational agents with memory and tool access

## >> Content Creation
- Agents that research , write, and fact-check content

## >> Decision Support
- Agents that gather data and provide recommendation